<a href="https://colab.research.google.com/github/PhuongAnhMannheim/Master/blob/master/schema_org_extract%26profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import json
import gzip
import pandas as pd
import tensorflow as tf
import sqlite3
from datetime import datetime as dt

In [0]:
!wget http://data.dws.informatik.uni-mannheim.de/structureddata/2018-12/quads/classspecific/md/schema_Review.gz

--2020-03-01 17:13:58--  http://data.dws.informatik.uni-mannheim.de/structureddata/2018-12/quads/classspecific/md/schema_Review.gz
Resolving data.dws.informatik.uni-mannheim.de (data.dws.informatik.uni-mannheim.de)... 134.155.95.56
Connecting to data.dws.informatik.uni-mannheim.de (data.dws.informatik.uni-mannheim.de)|134.155.95.56|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17837867803 (17G) [application/octet-stream]
Saving to: ‘schema_Review.gz.1’

schema_Review.gz.1   43%[=======>            ]   7.21G  47.6MB/s    eta 3m 33s 

In [36]:
conn = sqlite3.connect("reviews.db")
with conn: 
    conn.execute("""
        CREATE TABLE IF NOT EXISTS
            reviews(
                NODE TEXT,
                URL TEXT,bTEXT,
                REVIEWBODY TEXT,
                REVIEWRATING TEXT,
                ITEMREVIEWED TEXT,
                REVIEWASPECT TEXT,
                REVIEWAUTHOR TEXT, 
                REVIEWDESCR TEXT, 
                PRIMARY KEY (NODE, URL))
    """)
print("create reviews.db successful")

create reviews.db successful


In [30]:
# -> reviews.db -> converted to reviews_all.db
conn = sqlite3.connect("reviews.db")
cursor = conn.cursor()

taxo_pattern = re.compile("<http://schema.org/Review/reviewBody>|<http://schema.org/Review/ratingValue>|<http://schema.org/Review/itemReviewed>|<http://schema.org/Review/reviewAspect>|<http://schema.org/Review/author>|<http://schema.org/Review/description>|<http://schema.org/Review/datePublished>|<http://schema.org/Review/reviewRating>", re.IGNORECASE)
split_pattern = re.compile("^(_:.*)\s(<http:\/\/schema\.org\/Review\/reviewBody>|<http:\/\/schema\.org\/Review\/ratingValue>|<http:\/\/schema\.org\/Review\/itemReviewed>|<http:\/\/schema\.org\/Review\/reviewAspect>|<http:\/\/schema\.org\/Review\/author>|<http:\/\/schema\.org\/Review\/description>|<http:\/\/schema\.org\/Review\/datePublished>)\s(.*)\s<(.*)>\s\.$", re.IGNORECASE)

with gzip.open("schema_Review.gz", "rt") as f:
    i = 0
    skipped_lines = 0
    inserted = 0
    head = [next(f) for x in range(50)]
    # print(head)
    for line in head: 
      # print(line)
      if not taxo_pattern.search(line):
        # print("no taxo")
        continue
      match = split_pattern.match(line)
      if match is None:
        skipped_lines += 1
        print("no match" + str(line))  
        continue
      props = match.groups()
      with conn: 
        if len(props) != 4:
          print("properties with weird length: " + str(props))
          continue
        subject = props[0]
        predicate = props[1]
        obj = props[2]
        source = props[3]
        conn.execute("INSERT OR IGNORE INTO REVIEWS (NODE, URL) VALUES (?,?)",(subject, source))
        update_query = "UPDATE REVIEWS SET "
        params = [obj, subject, source]
        print(params)
        if predicate.lower() == "<http://schema.org/Review/reviewBody>".lower():
          update_query += "REVIEWBODY = ?"
        if predicate.lower() == "<http://schema.org/Review/ratingValue>".lower():
          update_query += "REVIEWRATING = ?"
        if predicate.lower() == "<http://schema.org/Review/itemReviewed>".lower():
          update_query += "ITEMREVIEWED = ?"
        if predicate.lower() == "<http://schema.org/Review/reviewAspect>".lower():
          update_query += "REVIEWASPECT = ?"
        if predicate.lower() == "<http://schema.org/Review/author>".lower():
          update_query += "REVIEWAUTHOR = ?"
        if predicate.lower() == "<http://schema.org/Review/description>".lower():
          update_query += "REVIEWDESCR = ?"
        if predicate.lower() == "<http://schema.org/Review/datePublished".lower():
          update_query += "REVIEWDATE = ?"
        if predicate.lower() == "<http://schema.org/Review/reviewRating".lower():
          update_query += "REVIEWRATING = ?"  
        update_query += " WHERE NODE = ? AND URL = ?;"
        try:
          print(update_query, params)
          conn.execute(update_query, params)
          print(props)
          inserted += 1
        except:
          print("failed to execute for params " + str(props))

print("inserted " + str(inserted) + " lines and skipped " + str(skipped_lines) + " lines")
print("Done processing the review file")




['<http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%B1-%DA%AF%D8%A7%D8%B2%DB%8C-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D8%B3%D8%B1%D8%A7%D8%B4%D9%BE%D8%B2/bz=1//ThemesStore/opencart/image/stars-0.png>', '_:node2c2e3ffd705962c891db9ab89a44cacc', 'http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%B1-%DA%AF%D8%A7%D8%B2%DB%8C-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D8%B3%D8%B1%D8%A7%D8%B4%D9%BE%D8%B2/bz=1']
UPDATE REVIEWS SET REVIEWRATING = ? WHERE NODE = ? AND URL = ?; ['<http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%B1-%DA%AF%D8%A7%D8%B2%DB%8C-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D8%B3%D8%B1%D8%A7%D8%B4%D9%BE%D8%B2/bz=1//ThemesStore/opencart/image/stars-0.png>', '_:node2c2e3ffd705962c891db9ab89a44cacc', 'http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%B1-%DA%AF%D8%A7%D8%B2%DB%8C-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D8%B3%D8%B1%D8%A7%D8%B4%D9%BE%D8%B2/bz=1']
('_:node2c2e3ffd705962c891db9ab89a44cacc', '<http://schema.org/Review/ratingValue>', '<http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%B1-%DA%AF%D8%A7%D8%B2%DB%8C-%D8%A

In [37]:
def total_rows(cursor, table_name, print_out=False):
    """ Returns the total number of rows in the database """
    cursor.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cursor.fetchall()
    if print_out:
        print('\nTotal rows: {}'.format(count[0][0]))
    return count[0][0]

total_rows(cursor, "REVIEWS")

2

In [38]:
# conn = sqlite3.connect("reviews.db")
# cursor = conn.cursor()
for row in cursor.execute("SELECT NODE, URL, REVIEWBODY, REVIEWRATING, ITEMREVIEWED, REVIEWASPECT, REVIEWAUTHOR, REVIEWDESCR from REVIEWS"):
  print("NODE = ", row[0])
  print("URL = ", row[1])
  print("REVIEWBODY = ", row[2])
  print("REVIEWRATING = ", row[3])
  print("ITEMREVIEWED = ", row[4])
  print("REVIEWASPECT = ", row[5])
  print("REVIEWAUTHOR = ", row[6])
  print("REVIEWDESCR = ", row[7], "\n")

print("done")

OperationalError: ignored

In [0]:
import findspark
findspark.init("C:\\Users\\D064966\\Downloads\\spark-3.0.0-preview-bin-hadoop2.7")

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, udf
from pyspark.sql.types import StringType
from langdetect import detect

spark = SparkSession \
    .builder \
    .appName("thesis") \
    .master("local[2]") \
    .config("spark.executor.memory", "4G") \
    .config("spark.driver.memory", "4G") \
    .config("spark.driver.maxResultSize", "4G") \
    .getOrCreate()

# filter out all reviews which do not have a review body and review rating
# total: 17,486,857
# without text: 17,211,204
# + without rating: 318,849

reviewsDF = spark.read.csv('../Data/reviews_all.csv', header=True, sep="\t")
print("total:" + str(reviewsDF.count()))
reviewsDF = reviewsDF.filter(reviewsDF.REVIEWBODY.isNotNull())
print("without text:" + str(reviewsDF.count()))
reviewsDF = reviewsDF.filter(reviewsDF.REVIEWRATING.isNotNull())
print("without rating:" + str(reviewsDF.count()))
reviewsDF.head(5)

# determine language of review body to distinct between english and non-english reviews
# with eng: 205,029

from langdetect import DetectorFactory
DetectorFactory.seed = 0

def detectLang(x):
    try:
        return detect(x)
    except:
        print("exception:", x)
    else: 
        print("sad:", x)
detect_lang = udf(detectLang, StringType())
df = reviewsDF.withColumn("LANG", lit(detect_lang(reviewsDF.REVIEWBODY)))

# amount of df = 318849
# df.head(5)
# df.count()

reviews_engDF = df.filter(df.LANG == "en")
print("with eng:" + str(reviews_engDF.count()))

# -> collection of csv in a new folder 
FilePath = "../Data/reviews_eng.csv"
# review_engDF.write.option("delimiter", "\t").format("com.databricks.spark.csv").save("../Data/reviews_eng.csv")
df.write\
    .mode("overwrite")\
    .format("com.databricks.spark.csv")\
    .option("header", "true")\
    .save(FilePath)


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving all.txt to all.txt
User uploaded file "all.txt" with length 200004262 bytes


In [0]:
colnames = ['NODE','URL','REVIEWBODY','REVIEWRATING','ITEMREVIEWED','REVIEWASPECT','REVIEWAUTHOR','REVIEWDESCR','LANG']
df_reviews_all = pd.read_csv('all.txt', names=colnames, sep="\n", engine="python")
df_reviews_all.head()

,NODE,URL,REVIEWBODY,REVIEWRATING,ITEMREVIEWED,REVIEWASPECT,REVIEWAUTHOR,REVIEWDESCR,LANG
0,"NODE,URL,REVIEWBODY,REVIEWRATING,ITEMREVIEWED,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"_:nodedb437f217f765d411af36362222b56c,http://w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"_:node1d8e8861dc9a25ed296bd922fd213dc,http://w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"_:node878dbbd41ab91c36d1c6a1fee5701a8d,https:/...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"_:node39e331755debb4c7295d6cff74fbf559,https:/...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df3 = df_reviews_all.NODE.str.split(",", n=1, expand=True)
df3.columns = ['NODE', 'URL']
df3.head()

ValueError: ignored

In [0]:
df_reviews_all[['URL', 'REVIEWBODY']] = df.URL.str.split(",", n=1, expand=True)
print(head)

In [0]:
df_reviews_all[['REVIEWBODY', 'LANG']] = df.REVIEWBODY.str.rsplit(",", 1)
print(head)

In [0]:
df_reviews_all[['REVIEWBODY', 'REVIEWDESCR']] = df.REVIEWBODY.str.rsplit(",", 1)